In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# 新たなモデルの実装

下記のモデル式を実現するモデルを実装する

$$
T = aXY + bXZ + cYZ + d
$$

T,X,Y,Z はそれぞれ関数コール回数・プロセス数・問題サイズ1・問題サイズ2
a,b,c は係数、dは切片

# TODOs

* ✅テストの作成
* ✅列名のリストにおいてモデル構築に必要な列の組み合わせを作成
* 実装


In [3]:
exp_names: list[str] = ["plot1", "plot2", "plot3", "plot4", "plot5"]
combinations_exp_name = list(itertools.combinations(exp_names, 2))
combinations_exp_name

[('plot1', 'plot2'),
 ('plot1', 'plot3'),
 ('plot1', 'plot4'),
 ('plot1', 'plot5'),
 ('plot2', 'plot3'),
 ('plot2', 'plot4'),
 ('plot2', 'plot5'),
 ('plot3', 'plot4'),
 ('plot3', 'plot5'),
 ('plot4', 'plot5')]

In [4]:
test_Model_LinearSumOf2elementCombination_ForMultipleRegression()